# (MBTI 별 Data 모으기: 생략) MBTI file 불러오기

In [25]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from collections import Counter

In [26]:
mbti=pd.read_csv('.\data\\training\\mbti.csv')

# UnderSampling

In [27]:
from imblearn.under_sampling import RandomUnderSampler

X_data = mbti['posts']
X = X_data.values.reshape(-1, 1)

y_data = mbti['type']
print(y_data)
y = y_data.values.reshape(-1, 1)
print(y)

X_resampled, y_resampled = RandomUnderSampler(random_state=0).fit_sample(X, y)
#format(Counter(y_resampled))
X_resampled #ndarray
Counter(y_resampled)

0       ENFJ
1       ENFJ
2       ENFJ
3       ENFJ
4       ENFJ
        ... 
8670    ISTP
8671    ISTP
8672    ISTP
8673    ISTP
8674    ISTP
Name: type, Length: 8675, dtype: object
[['ENFJ']
 ['ENFJ']
 ['ENFJ']
 ...
 ['ISTP']
 ['ISTP']
 ['ISTP']]


Counter({'ENFJ': 39,
         'ENFP': 39,
         'ENTJ': 39,
         'ENTP': 39,
         'ESFJ': 39,
         'ESFP': 39,
         'ESTJ': 39,
         'ESTP': 39,
         'INFJ': 39,
         'INFP': 39,
         'INTJ': 39,
         'INTP': 39,
         'ISFJ': 39,
         'ISFP': 39,
         'ISTJ': 39,
         'ISTP': 39})

# Type value 

In [28]:
b_Types = {'I':0, 'E':1, 'N':0, "S":1, 'F':0, 'T':1, 'J':0, 'P':1}
b_Types_list =[{0:'I', 1:'E'}, {0:'N', 1:'S'}, {0:'F', 1:'T'}, {0:'J', 1:'P'}]

def translate_personality(personality):
    return[b_Types[i] for i in personality]

def translate_back(personality):
    s = ""
    for i, j in enumerate(personality):
        s += b_Types_list[i][j]
    return s

d = y[0]
print(d)
list_personality_b = np.array([translate_personality(p) for p in d])
print(list_personality_b)

['ENFJ']
[[1 0 0 0]]


# tokenize and stemming

In [29]:
X_data = X_resampled.ravel()
y_data = y_resampled


print("normalize 시작")
normalized_text = []
for sentence in X_data: #for sentence in X_data.ravel()
    rm_urls = re.sub(r'http[s]?://\S+', '', sentence)
    clean_sentence = re.sub('[^A-Za-z\s]+', '', rm_urls.lower())
    normalized_text.append(clean_sentence)
X_data = normalized_text
print("normalize 완료")

normalize 시작
normalize 완료


In [30]:
import nltk
import gc
# nltk.download('punkt')
# nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [31]:
ps=PorterStemmer()

# print("stopwords 제거 시작")
stop_words = set(stopwords.words('english'))
#print("stopwords 제거 완료")

print("stemming 시작")
stemData=[]
for sentence in X_data:
    tokenData = nltk.word_tokenize(sentence)
    tempData = []
    for word in tokenData:
        if word not in stop_words:
            tempData.append(ps.stem(word))
    stemData.append(tempData)
print("stemming 완료")
print(stemData[0])
stemData = np.array(stemData)
#'list' object has no attribute 'lower' 문제 발생
#2차원 리스트인 stemData를 1차원 리스트로
#flat_stem = [item for sublist in stemData for item in sublist]
del [[mbti]]
gc.collect()

stemming 시작
stemming 완료
['want', 'know', 'feel', 'intp', 'actual', 'intp', 'idea', 'intp', 'havent', 'met', 'anyten', 'million', 'hug', 'mani', 'amaz', 'convers', 'held', 'absenc', 'vow', 'never', 'leav', 'realli', 'lot', 'written', 'recent', 'terriblyfacebook', 'way', 'emphas', 'other', 'error', 'cognit', 'becom', 'obviou', 'lead', 'wonder', 'im', 'friend', 'mani', 'peopl', 'rememb', 'peopl', 'moremmmmhm', 'ive', 'wonder', 'late', 'ei', 'think', 'definit', 'much', 'needwant', 'compani', 'around', 'regardless', 'time', 'togeth', 'spenthahah', 'love', 'think', 'that', 'go', 'topic', 'excel', 'convers', 'thank', 'particip', 'indulg', 'interest', 'enlighteningsomeon', 'creat', 'chitchat', 'thread', 'enfj', 'forum', 'enfj', 'vigilantli', 'await', 'type', 'wander', 'start', 'conversationswa', 'enfp', 'ive', 'caught', 'mani', 'text', 'look', 'around', 'shini', 'object', 'give', 'midsyl', 'mm', 'hmm', 'alway', 'mani', 'andi', 'tire', 'extravert', 'energysap', 'sentiment', 'arent', 'batshit', 

196

# Vectorization

In [32]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [33]:
count_vectorizer = CountVectorizer(
    analyzer="word",
    max_features=1500,
    tokenizer=None,
    stop_words=None,
    max_df=0.7,
    min_df=0.1
)

#X_ct = count_vectorizer.fit_transform(stemData)

def dummy_fun(doc):
    return doc

tfidf=TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None
)

tfidf.fit(stemData)
print(tfidf.transform(stemData).toarray())
X = tfidf.transform(stemData)
print(X)
y = np.array(y_data)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.05858548 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.04542902 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
  (0, 38616)	0.031996512659156894
  (0, 38578)	0.03286039560761366
  (0, 38565)	0.057208490852473474
  (0, 38556)	0.06369299789656549
  (0, 38482)	0.07370640937171992
  (0, 38432)	0.023110327077128505
  (0, 38411)	0.056118144533601276
  (0, 38346)	0.030911604753450596
  (0, 38304)	0.050319331085647706
  (0, 38051)	0.03852987969548263
  (0, 37982)	0.021758208967633117
  (0, 37964)	0.035121536565144516
  (0, 37804)	0.03987971435535688
  (0, 37774)	0.02453157411311739
  (0, 37752)	0.04127966139118793
  (0, 37742)	0.028172244380434894
  (0, 37325)	0.0661

# XGBoost_Undersampling

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state=1234)


# Model Building & Evaluation

In [35]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from matplotlib import pyplot

In [36]:
print("Model building Start")
#print(max(len(l) for l in X_train))
model = XGBClassifier()
model.fit(X_train, y_train, verbose=1)
#model.get_booster().get_score(importance_type="gain") #변수의 중요도

Model building Start


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [37]:
import xgboost as xgb
#test_hp = xgb.DMatrix(data=VHX_data, label=hy)
preds = model.predict(X_test)
print("Model building End")

Model building End


In [38]:
import joblib

joblib.dump(model, 'XGBoost_undersampleHOXY.pkl')

['XGBoost_undersampleHOXY.pkl']

In [39]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

print("Evaluation Start")
# labels과 guesses
labels = preds
guesses = y_test

print(accuracy_score(labels, guesses))
print(recall_score(labels, guesses, average=None))
print(precision_score(labels, guesses, average=None))
print(f1_score(labels, guesses, average=None))

print("Evaluation END")
# https://eunsukimme.github.io/ml/2019/10/21/Accuracy-Recall-Precision-F1-score/

Evaluation Start
0.4946808510638298
[0.71428571 0.4        0.27272727 0.23529412 0.81818182 0.41666667
 0.54545455 1.         0.33333333 0.7        0.375      0.16666667
 0.5        0.46666667 0.61111111 0.58333333]
[0.625      0.30769231 0.375      0.4        0.5625     0.45454545
 0.4        0.61538462 0.25       0.53846154 0.33333333 0.2
 0.66666667 0.7        0.61111111 0.63636364]
[0.66666667 0.34782609 0.31578947 0.2962963  0.66666667 0.43478261
 0.46153846 0.76190476 0.28571429 0.60869565 0.35294118 0.18181818
 0.57142857 0.56       0.61111111 0.60869565]
Evaluation END


# Harry Potter

In [40]:
import nltk
import numpy as np
import pandas as pd

from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

import re
import glob   

'''HP data Normalization and Stemming'''
# bag_of_words = {} #{등장인물 : 해당인물의 stemData, ~}
h_normalized_text = []
h_stemData = []

ps=PorterStemmer()
stop_words = set(stopwords.words('english'))
input_file = r'.\data\\test\\'

allHPfile_list = glob.glob(os.path.join(input_file, 'HP_*'))
print(allHPfile_list)

harryPotter={}
for file in allHPfile_list:
    df = pd.read_csv(file, sep=',', encoding='iso-8859-1')
    harryPotter[df['Character'][0]] = df['Sentence'].ravel()

print("stemming 시작")
for element in harryPotter.keys():
    for sentence in harryPotter[element]:
        h_clean_sentence = re.sub('[^A-Za-z\s]+', '', sentence.lower())
        h_normalized_text.append(h_clean_sentence)
        for sentence in h_normalized_text:
            tokenData = nltk.word_tokenize(sentence)
            for word in tokenData:
                if word not in stop_words:
                    h_stemData.append(ps.stem(word))
    harryPotter[element] = h_stemData

print("stemming 완료")

['.\\data\\\\test\\HP_DUMBLEDORE.csv', '.\\data\\\\test\\HP_HAGRID.csv', '.\\data\\\\test\\HP_HARRY.csv', '.\\data\\\\test\\HP_HERMIONE.csv', '.\\data\\\\test\\HP_RON.csv']
stemming 시작
stemming 완료


In [41]:
'''HP data Vectorization'''
# import numpy as np
# from sklearn.feature_extraction.text import TfidfVectorizer

stemData = []
hy=[]
for element in harryPotter.keys():
    stemData.append(harryPotter[element])
    hy.append(element)
HX_data = tfidf.transform(stemData)
print(HX_data)
HY_data = np.array(hy)

'''for element in harryPotter.keys():
    tfidf.fit(harryPotter[element])
    script = tfidf.transform(harryPotter[element])
    X.append(script)
    y.append(element)

print(X.shape)'''

  (0, 38777)	0.0013169134088659989
  (0, 38733)	0.027392104038965377
  (0, 38708)	0.0086032637148345
  (0, 38616)	0.057307869763401256
  (0, 38587)	0.0015145477386770783
  (0, 38578)	0.015677904024057377
  (0, 38565)	0.035331674498240935
  (0, 38502)	0.011336388220755942
  (0, 38432)	0.004425140166460562
  (0, 38423)	0.0011010479203341467
  (0, 38400)	0.006040401794255721
  (0, 38395)	0.0035042319995763132
  (0, 38359)	0.010278407406606448
  (0, 38346)	0.031576821869970045
  (0, 38328)	0.025968159038507896
  (0, 38324)	0.04338432539669251
  (0, 38289)	0.006678807481014122
  (0, 38083)	0.00748275380908028
  (0, 38055)	0.009590532552081355
  (0, 38051)	0.008782777986629348
  (0, 38011)	1.2650613906402689e-05
  (0, 37999)	0.003426623993556433
  (0, 37982)	0.005140786300581928
  (0, 37964)	0.037196378622580975
  (0, 37954)	0.00017487450343866594
  :	:
  (4, 691)	0.015495337032055372
  (4, 669)	0.04230744322019207
  (4, 648)	0.0030658006668705133
  (4, 629)	0.011534367632245093
  (4, 541)	0

'for element in harryPotter.keys():\n    tfidf.fit(harryPotter[element])\n    script = tfidf.transform(harryPotter[element])\n    X.append(script)\n    y.append(element)\n\nprint(X.shape)'

# Predict & Get Results

In [49]:
import joblib  
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from matplotlib import pyplot

model = joblib.load('XGBoost_undersample.pkl')
# xgb_model_loaded.get_xgb_params()    # Get xgboost specific parameters.
harrypotter_predict = model.predict(HX_data)
print(harrypotter_predict)

['ENTP' 'ENTP' 'ENTP' 'ENTP' 'ENTP']


In [47]:
cols_when_model_builds = model.get_booster().feature_names
len(cols_when_model_builds)

295647